In [15]:
import chess.engine
import chess 
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import chess.pgn 
from stockfish import Stockfish
import pandas as pd
import time
import numpy as np
import chess
import chess.pgn
import time
import multiprocessing as mp
import sys
from stockfish import Stockfish
import numpy as np
import json

In [16]:
from maia_functions import *

In [22]:
df = pd.read_json('../data/curated/FIDE_MAIA.ndjson', lines = True)

In [41]:
df.shape

(40030, 35)

In [32]:
# Apply the lambda function to each element
mask = df['maia_cpl_w'].apply(lambda x: not isinstance(x, (list, str)) or len(x) <= 0)

other_mask = df['stockfish_eval'].apply(lambda x: len(x) > 7)

# Find the indices where any element in the row satisfies the condition
invalid_indices = df[mask].index

In [35]:
mask_indices = df[mask].index
other_mask_indices = df[other_mask].index
intersection_indices = mask_indices.intersection(other_mask_indices)

In [38]:
intersection_indices[501:510]

Index([4098, 4099, 4136, 4137, 4233, 4234, 4235, 4236, 4243], dtype='int64')

In [39]:
df.iloc[4098] # Aizharyk, Kaliza

event                    FIDE World Youth Rapid Champ - Asia - U12 - Girls
round                                                                 2.14
white                                                         Das, Arshiya
black                                                        Lee, Xia Sean
result                                                                 1-0
moves                    [e2e4, e7e5, g1f3, b8c6, f1b5, a7a6, b5c6, d7c...
white_elo                                                             1275
black_elo                                                             1016
ECO                                                                    C69
Opening                                                                   
white_cpl                [23, 7, 11, -2, -12, 28, 36, 251, -18, -16, 3,...
black_cpl                [2, -12, -8, 12, 18, 28, 262, 313, 31, 73, 20,...
stockfish_eval           [9, 9, 9, 9, 7, 7, 9, 9, -10, -10, -17, -17, -...
queen_moved_at           

In [18]:
print(len(df))
df = df[df['white'] != "?"]
df = df[df['black'] != "?"]

print(len(df))
df = df[df['stockfish_eval'].apply(lambda x: len(x) > 7)]
print(len(df))
df_temp=df[df['maia_cpl_w'].apply(lambda x: not isinstance(x, (list, str)) or len(x) <= 0)]

40030
32506
28934


In [21]:
df_temp

,event,round,white,black,result,moves,white_elo,black_elo,ECO,Opening,...,end_K,end_n,end_q,end_B,end_N,promotion,can_claim_draw,insufficient_material,maia_cpl_w,maia_cpl_b
500,FIDE Online Olympiad for people with disabilities,4.16.3,"Lago Carballo, Carmen","Malika, Handa",1/2-1/2,"[e2e4, c7c5, g1f3, b8c6, f1c4, g7g6, d2d3, f8g...",1328,1345,B30,Sicilian defence,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
501,FIDE Online Olympiad for people with disabilities,4.16.4,"Bhargav Tokala, Tarun","Fernandez Lago, David",1/2-1/2,"[e2e4, c7c5, g1f3, e7e6, d2d4, c5d4, f3d4, g8f...",1025,1958,B40,Sicilian,...,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,[],[]
502,FIDE Online Olympiad for people with disabilities,4.17.1,"Barata, Adriano Albiani","Redor, Menandro",1-0,"[e2e4, d7d5, e4d5, g8f6, g1f3, f6d5, d2d4, c8f...",2144,2019,B01,Scandinavian defence,...,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,[],[]
504,FIDE Online Olympiad for people with disabilities,4.17.3,"Lopes, Marcia Maria Dias","Tellor, Max Dave",0-1,"[d2d4, g8f6, c1f4, d7d5, g1f3, c8g4, e2e3, e7e...",1363,1771,D02,Queen's bishop game,...,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,[],[]
506,FIDE Online Olympiad for people with disabilities,4.18.1,"Soltanov, Serik","Osorio, Jesus Camilo",1/2-1/2,"[d2d4, d7d5, g1f3, g8f6, c1f4, c7c6, e2e3, c8g...",2266,1859,D02,Queen's bishop game,...,1.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34987,Youth Grand Prix 1 - Open - Open 18,2.1,"Makoveev, Ilya","Qalichava, Akaki",1-0,"[e2e4, c7c5, g1f3, e7e6, d2d4, c5d4, f3d4, b8c...",2322,2042,B48,Sicilian,...,1.0,1.0,1.0,NaN,1.0,NaN,NaN,NaN,[],[]
34988,Youth Grand Prix 1 - Open - Open 18,2.2,"Ilyasli, Ughur","Liyanage, Ranindu Dilshan",1/2-1/2,"[e2e4, c7c5, g1f3, d7d6, d2d4, c5d4, d1d4, b8c...",2341,1926,B53,"Sicilian, Chekhover variation",...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
34989,Youth Grand Prix 1 - Open - Open 18,2.3,"George Samir, David","Thilakarathne, G M H",1-0,"[e2e4, c7c5, g1f3, d7d6, d2d4, c5d4, f3d4, g8f...",1804,2398,B90,Sicilian,...,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,[],[]
34992,Youth Grand Prix 1 - Open - Open 18,3.1,"Okeke, Isaac Chukwudalu","Makoveev, Ilya",1/2-1/2,"[e2e4, c7c5, g1f3, d7d6, d2d4, c5d4, f3d4, g8f...",2179,2322,B90,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]


In [6]:
df_dict = df_temp.to_dict(orient='records')

In [13]:
df_temp.columns

Index(['event', 'round', 'white', 'black', 'result', 'moves', 'white_elo',
       'black_elo', 'ECO', 'Opening', 'white_cpl', 'black_cpl',
       'stockfish_eval', 'queen_moved_at', 'queen_changed_at', 'total_checks',
       'first_check_at', 'total_moves', 'end_r', 'end_k', 'end_p', 'end_Q',
       'end_b', 'end_P', 'end_R', 'end_K', 'end_n', 'end_q', 'end_B', 'end_N',
       'promotion', 'can_claim_draw', 'insufficient_material', 'maia_cpl_w',
       'maia_cpl_b'],
      dtype='object')

In [14]:
for game in df_dict:
    stockfish=Stockfish("../stockfish/stockfish-windows-x86-64-avx2.exe")
    stockfish.set_depth(15)#How deep the AI looks
    mm = maia_cpl(game, stockfish)
    game["maia_cpl_w"] = mm[0]
    game["maia_cpl_b"] = mm[1]
    break

<UciProtocol (pid=11988)>: stderr >>        _
<UciProtocol (pid=11988)>: stderr >> |   _ | |
<UciProtocol (pid=11988)>: stderr >> |_ |_ |_| v0.31.1 built Aug 11 2024
<UciProtocol (pid=11988)>: stderr >> Found configuration file: c:\pc_coding\Finding-Elo\maia.chess\lc0_windows/lc0.config
<UciProtocol (pid=10716)>: stderr >>        _
<UciProtocol (pid=10716)>: stderr >> |   _ | |
<UciProtocol (pid=10716)>: stderr >> |_ |_ |_| v0.31.1 built Aug 11 2024
<UciProtocol (pid=10716)>: stderr >> Found configuration file: c:\pc_coding\Finding-Elo\maia.chess\lc0_windows/lc0.config
<UciProtocol (pid=11988)>: stderr >> Loading weights file from: lc0_windows\models\maia-1300.pb.gz
<UciProtocol (pid=11988)>: stderr >> Creating backend [cuda-auto]...
<UciProtocol (pid=11988)>: stderr >> Switching to [cuda-fp16]...
<UciProtocol (pid=11988)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=11988)>: stderr >> Latest version of CUDA supported by the driver: 12.6.0
<UciProtocol (pid=11988)>: stderr

--weights=lc0_windows\models\maia-1300.pb.gz
--weights=lc0_windows\models\maia-1300.pb.gz


<UciProtocol (pid=10716)>: stderr >> Loading weights file from: lc0_windows\models\maia-1300.pb.gz
<UciProtocol (pid=10716)>: stderr >> Creating backend [cuda-auto]...
<UciProtocol (pid=10716)>: stderr >> Switching to [cuda-fp16]...
<UciProtocol (pid=10716)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=10716)>: stderr >> Latest version of CUDA supported by the driver: 12.6.0
<UciProtocol (pid=10716)>: stderr >> GPU: NVIDIA GeForce RTX 3070
<UciProtocol (pid=10716)>: stderr >> GPU memory: 7.99951 Gb
<UciProtocol (pid=10716)>: stderr >> GPU clock frequency: 1815 MHz
<UciProtocol (pid=10716)>: stderr >> GPU compute capability: 8.6
<UciProtocol (pid=10716)>: stderr >> L2 cache capacity: 4194304


In [12]:
mm[1]

[0,
 -4,
 0,
 16,
 0,
 -2,
 0,
 0,
 0,
 -6,
 5,
 0,
 0,
 15,
 6,
 -3,
 0,
 0,
 -8,
 0,
 0,
 -35,
 -26,
 -14,
 -9,
 -27,
 -19,
 0,
 -8,
 -1,
 0,
 -17]

In [10]:
df_temp.head()

,event,round,white,black,result,moves,white_elo,black_elo,ECO,Opening,...,end_K,end_n,end_q,end_B,end_N,promotion,can_claim_draw,insufficient_material,maia_cpl_w,maia_cpl_b
500,FIDE Online Olympiad for people with disabilities,4.16.3,"Lago Carballo, Carmen","Malika, Handa",1/2-1/2,"[e2e4, c7c5, g1f3, b8c6, f1c4, g7g6, d2d3, f8g...",1328,1345,B30,Sicilian defence,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
501,FIDE Online Olympiad for people with disabilities,4.16.4,"Bhargav Tokala, Tarun","Fernandez Lago, David",1/2-1/2,"[e2e4, c7c5, g1f3, e7e6, d2d4, c5d4, f3d4, g8f...",1025,1958,B40,Sicilian,...,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,[],[]
502,FIDE Online Olympiad for people with disabilities,4.17.1,"Barata, Adriano Albiani","Redor, Menandro",1-0,"[e2e4, d7d5, e4d5, g8f6, g1f3, f6d5, d2d4, c8f...",2144,2019,B01,Scandinavian defence,...,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,[],[]
504,FIDE Online Olympiad for people with disabilities,4.17.3,"Lopes, Marcia Maria Dias","Tellor, Max Dave",0-1,"[d2d4, g8f6, c1f4, d7d5, g1f3, c8g4, e2e3, e7e...",1363,1771,D02,Queen's bishop game,...,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,[],[]
506,FIDE Online Olympiad for people with disabilities,4.18.1,"Soltanov, Serik","Osorio, Jesus Camilo",1/2-1/2,"[d2d4, d7d5, g1f3, g8f6, c1f4, c7c6, e2e3, c8g...",2266,1859,D02,Queen's bishop game,...,1.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,[],[]


In [3]:
def extract_features(game):
    # Read the next game
    features = defaultdict(int)
    stockfish=Stockfish("../stockfish/stockfish-windows-x86-64-avx2.exe")
    stockfish.set_depth(15)#How deep the AI looks
    features = defaultdict(int)
    features["white"] = game.headers.get('White', '')
    features["black"] = game.headers.get('Black', '')
    features["moves"] = [str(move) for move in game.mainline_moves()]
    features["WhiteElo"] = game.headers.get('WhiteElo', '')
    features["BlackElo"] = game.headers.get('BlackElo', '')
    
    try:
        mm = maia_cpl(features, stockfish)
        features["maia_cpl_w"] = mm[0]
        features["maia_cpl_b"] = mm[1]
    except Exception as e:
        print(e)
        return features
    return features

In [4]:
pgn_file_path = "../data/landing/fics_openings.pgn"

games = []
pgn_file = open(pgn_file_path)
game = chess.pgn.read_game(pgn_file)

start = 0
limit =6000
count = 0
while game is not None:
    #features = extract_features(game)
    games.append(game)
    
    count += 1
    if count == limit:
        break
    game = chess.pgn.read_game(pgn_file)

pgn_file.close()

In [5]:
chunk_size = 1000  # Process 1000 games at a time
num_chunks = len(games) // chunk_size + (1 if len(games) % chunk_size > 0 else 0)
for i in range(4, 40):

    start_index = i * chunk_size
    end_index = min(start_index + chunk_size, len(games))
    games_chunk = games[start_index:end_index]

    try:
        result = []
        for game in games_chunk:
            features = extract_features(game)
            result.append(features)

        ndjson_file_path = f'../data/raw/maia/fics_{start_index}.ndjson'
        print(f"saved to {ndjson_file_path}")
        print("--------------------------------------------------------")
        with open(ndjson_file_path, 'w') as f:
            for item in result:
                json.dump(item, f)
                f.write('\n') 
            f.close()
    except Exception as e:
       # By this way we can know about the type of error occurring
        print("The error is: ",e)
    break
      

--weights=lc0_windows\models\maia-1200.pb.gz


<UciProtocol (pid=19332)>: stderr >>        _
<UciProtocol (pid=19332)>: stderr >> |   _ | |
<UciProtocol (pid=19332)>: stderr >> |_ |_ |_| v0.31.1 built Aug 11 2024
<UciProtocol (pid=19332)>: stderr >> Found configuration file: c:\pc_coding\Finding-Elo\maia.chess\lc0_windows/lc0.config
<UciProtocol (pid=10664)>: stderr >>        _
<UciProtocol (pid=10664)>: stderr >> |   _ | |
<UciProtocol (pid=10664)>: stderr >> |_ |_ |_| v0.31.1 built Aug 11 2024
<UciProtocol (pid=10664)>: stderr >> Found configuration file: c:\pc_coding\Finding-Elo\maia.chess\lc0_windows/lc0.config
<UciProtocol (pid=19332)>: stderr >> Loading weights file from: lc0_windows\models\maia-1200.pb.gz
<UciProtocol (pid=19332)>: stderr >> Creating backend [cuda-auto]...
<UciProtocol (pid=19332)>: stderr >> Switching to [cuda-fp16]...
<UciProtocol (pid=19332)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=19332)>: stderr >> Latest version of CUDA supported by the driver: 12.6.0
<UciProtocol (pid=19332)>: stderr

--weights=lc0_windows\models\maia-1100.pb.gz


<UciProtocol (pid=10664)>: stderr >> Loading weights file from: lc0_windows\models\maia-1100.pb.gz
<UciProtocol (pid=10664)>: stderr >> Creating backend [cuda-auto]...
<UciProtocol (pid=10664)>: stderr >> Switching to [cuda-fp16]...
<UciProtocol (pid=10664)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=10664)>: stderr >> Latest version of CUDA supported by the driver: 12.6.0
<UciProtocol (pid=10664)>: stderr >> GPU: NVIDIA GeForce RTX 3070
<UciProtocol (pid=10664)>: stderr >> GPU memory: 7.99951 Gb
<UciProtocol (pid=10664)>: stderr >> GPU clock frequency: 1815 MHz
<UciProtocol (pid=10664)>: stderr >> GPU compute capability: 8.6
<UciProtocol (pid=10664)>: stderr >> L2 cache capacity: 4194304
<UciProtocol (pid=20680)>: stderr >>        _
<UciProtocol (pid=20680)>: stderr >> |   _ | |
<UciProtocol (pid=20680)>: stderr >> |_ |_ |_| v0.31.1 built Aug 11 2024
<UciProtocol (pid=20680)>: stderr >> Found configuration file: c:\pc_coding\Finding-Elo\maia.chess\lc0_windows/lc0.config

--weights=lc0_windows\models\maia-1800.pb.gz
--weights=lc0_windows\models\maia-1700.pb.gz


...

--weights=lc0_windows\models\maia-1600.pb.gz
--weights=lc0_windows\models\maia-1600.pb.gz


<UciProtocol (pid=14640)>: stderr >> Loading weights file from: lc0_windows\models\maia-1600.pb.gz
<UciProtocol (pid=14640)>: stderr >> Creating backend [cuda-auto]...
<UciProtocol (pid=14640)>: stderr >> Switching to [cuda-fp16]...
<UciProtocol (pid=14640)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=14640)>: stderr >> Latest version of CUDA supported by the driver: 12.6.0
<UciProtocol (pid=14640)>: stderr >> GPU: NVIDIA GeForce RTX 3070
<UciProtocol (pid=14640)>: stderr >> GPU memory: 7.99951 Gb
<UciProtocol (pid=14640)>: stderr >> GPU clock frequency: 1815 MHz
<UciProtocol (pid=14640)>: stderr >> GPU compute capability: 8.6
<UciProtocol (pid=14640)>: stderr >> L2 cache capacity: 4194304
<UciProtocol (pid=9924)>: stderr >>        _
<UciProtocol (pid=9924)>: stderr >> |   _ | |
<UciProtocol (pid=9924)>: stderr >> |_ |_ |_| v0.31.1 built Aug 11 2024
<UciProtocol (pid=9924)>: stderr >> Found configuration file: c:\pc_coding\Finding-Elo\maia.chess\lc0_windows/lc0.config
<Uc

--weights=lc0_windows\models\maia-1400.pb.gz
--weights=lc0_windows\models\maia-1300.pb.gz


<UciProtocol (pid=21392)>: stderr >> Loading weights file from: lc0_windows\models\maia-1300.pb.gz
<UciProtocol (pid=21392)>: stderr >> Creating backend [cuda-auto]...
<UciProtocol (pid=21392)>: stderr >> Switching to [cuda-fp16]...
<UciProtocol (pid=21392)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=21392)>: stderr >> Latest version of CUDA supported by the driver: 12.6.0
<UciProtocol (pid=21392)>: stderr >> GPU: NVIDIA GeForce RTX 3070
<UciProtocol (pid=21392)>: stderr >> GPU memory: 7.99951 Gb
<UciProtocol (pid=21392)>: stderr >> GPU clock frequency: 1815 MHz
<UciProtocol (pid=21392)>: stderr >> GPU compute capability: 8.6
<UciProtocol (pid=21392)>: stderr >> L2 cache capacity: 4194304


saved to ../data/raw/maia/fics_4000.ndjson
--------------------------------------------------------
